In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from src.configurations import Configuration
from src.hmm import HMMEventDetector
from src.features import FeatureSet
from src.helper import load_final_filtered_csv
from src.night_analysis import NightAnalyser

2025-06-20 14:43:12.237 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Users\ross\OneDrive\Documents\Masters\Project\masters_project


In [2]:
config = Configuration()
df = load_final_filtered_csv(config, interpolate_cob=True)
night_level_features = ['iob mean', 'cob mean', 'bg mean',
                        'iob max', 'cob max']
analyser = NightAnalyser(df=df[night_level_features],
                         feature_settings='custom')
analyser.extract_night_level_features(night_start_hour=17)
analyser.preprocess_night_features(n_components=2)
analyser.cluster_nights(print_clusters=False, plot_2d=False)

['iob mean', 'cob mean', 'bg mean', 'iob max', 'cob max']
Extracting night-level features using NoneType settings...


C:\Users\ross\OneDrive\Documents\Masters\Project\masters_project\src\night_analysis.py:148: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  night_start_date = temp_df['datetime'].dt.date - pd.to_timedelta((hour < night_start_hour).astype(int), unit='D')
Feature Extraction: 100%|██████████| 30/30 [00:05<00:00,  5.89it/s]


Extracted 68 features for 236 nights.
Preprocessing night-level features (scaling and PCA)...
PCA reduced dimensions from 65 to 2.
Clustering nights into 3 clusters...


array([0, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2,
       1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2,
       1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [3]:
features = FeatureSet(df=df, sample_rate=30)
df = features.get_all_features()
df = analyser.return_dataset_with_clusters(df[analyser.])  # Add the night cluster labels to the df
mean_cols = ['iob mean','cob mean','bg mean']
print()

Scaling ['iob mean', 'cob mean', 'bg mean', 'iob min', 'cob min', 'bg min', 'iob max', 'cob max', 'bg max', 'day_type_weekday', 'day_type_weekend', 'iob mean_rate_of_change', 'cob mean_rate_of_change', 'bg mean_rate_of_change', 'iob mean hourly_mean', 'cob mean hourly_mean', 'bg mean hourly_mean', 'iob max_peaks_above_mean', 'cob max_peaks_above_mean', 'bg max_peaks_above_mean', 'hour_of_day', 'hour_sin', 'hour_cos'] columns


In [9]:
# --- HMM Event Detection ---
print("\n--- HMM Event Detection Example ---")
hmm_detector = HMMEventDetector(df, features.get_all_feature_columns(),
                                 original_var_cols=mean_cols,)
hmm_detector.train_model(n_components=3)

# Visualize for a specific individual
some_individual_id = df.index.get_level_values('id').unique()[0]
print(some_individual_id)
hmm_detector.visualize_individual_events(some_individual_id)

# Visualize for a specific night cluster (e.g., cluster 0)
some_cluster_label = df['cluster_label'].unique()[0]
hmm_detector.visualize_cluster_events(some_cluster_label)


--- HMM Event Detection Example ---
Training HMM with 3 components...
[[0.27005727 0.         0.6233895  ... 0.7826087  0.         0.46410162]
 [0.28513173 0.325      0.60220994 ... 0.7826087  0.         0.46410162]
 [0.37255441 0.29583333 0.61740331 ... 0.82608696 0.01703709 0.60280232]
 ...
 [0.13713631 0.         0.29005525 ... 0.39130435 0.85355339 0.08516423]
 [0.1347079  0.         0.28176796 ... 0.43478261 0.75       0.        ]
 [0.12962199 0.         0.23314917 ... 0.43478261 0.75       0.        ]]
HMM training complete.
Learned Transition Matrix:
 [[0.75710255 0.20238755 0.04050989]
 [0.38543127 0.5603491  0.05421963]
 [0.04104796 0.20336366 0.75558838]]

Learned Means of States:
 [[-0.39559881 -0.49300821 -0.2050724  -0.36591334 -0.39387076 -0.18307524
  -0.40979945 -0.53087579 -0.2221468  -0.04920434  0.04920434  0.09959875
   0.2186694   0.09887857 -0.32558327 -0.41440257 -0.18123331 -0.3653366
  -0.31933761 -0.34325869 -0.16152865  0.30662867 -0.24579985]
 [-0.09874175 

KeyError: 'id'